In [1]:
import ast
import inspect
from pathlib import Path

In [124]:
def eval_str(node) -> str:
    return node.s

def eval_name(node) -> str:
    return node.id


def eval_index(node):
    return eval_node(node.value)


def eval_tuple(node):
    return ', '.join((eval_node(item) for item in node.elts))


def eval_attribute(node):
    return f'{eval_node(node.value)}.{node.attr}'


def eval_node(node):
    node_class = type(node)
    return EVAL_MAP[node_class](node)


def eval_subscript(node) -> str:
    return f'{eval_node(node.value)}[{eval_node(node.slice)}]'


EVAL_MAP = {ast.Name: eval_name,
            ast.Index: eval_index,
            ast.Tuple: eval_tuple,
            ast.Attribute: eval_attribute,
            ast.Subscript: eval_subscript,
            ast.Str: eval_str}

In [125]:
# file = Path('equation_parser/from_ast.py')
file = Path('equation_parser/functions.py')

raw_tree = file.read_text()
tree = ast.parse(raw_tree)

In [126]:
ast.dump(tree)

"Module(body=[Expr(value=Str(s='Custom function in the parser')), ImportFrom(module='typing', names=[alias(name='TYPE_CHECKING', asname=None), alias(name='Union', asname=None)], level=0), If(test=Name(id='TYPE_CHECKING', ctx=Load()), body=[Import(names=[alias(name='pandas', asname='pd')])], orelse=[]), FunctionDef(name='identity', args=arguments(args=[arg(arg='x', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Str(s='Convenience function returning the input value')), Return(value=Name(id='x', ctx=Load()))], decorator_list=[], returns=None), FunctionDef(name='_get_ref_values', args=arguments(args=[arg(arg='values', annotation=Str(s='Union[pd.DataFrame, pd.Series]')), arg(arg='ref_id', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Str(s='Get reference from values either as scalar or as numpy array')), Assign(targets=[Name(id='ref_values', ctx=Store())], value=Subscript(v

In [127]:
classes = [item for item in tree.body if isinstance(item, ast.ClassDef)]

class_methods = [func for item in classes for func in item.body if isinstance(func, ast.FunctionDef)]

functions = [item for item in tree.body if isinstance(item, ast.FunctionDef)]

In [132]:
for func in class_methods:
    print(f'def {func.name}():')
    if ast.get_docstring(func) is not None:
        print(f'\t{ast.get_docstring(func)}')
        for item in func.args.args:
            if item.annotation is not None:
                print(f'\t{item.arg}: {eval_node(item.annotation)}')
    print()

In [133]:
for func in functions:
    print(f'def {func.name}():')
    if ast.get_docstring(func) is not None:
        print(f'\t{ast.get_docstring(func)}')
        for item in func.args.args:
            if item.annotation is not None:
                print(f'\t{item.arg}: {eval_node(item.annotation)}')
    print()

def identity():
	Convenience function returning the input value

def _get_ref_values():
	Get reference from values either as scalar or as numpy array
	values: Union[pd.DataFrame, pd.Series]

def relative_percentage():
	Compute the percentage of varation of values relative to the value in ref_id
	values: Union[pd.DataFrame, pd.Series]

def delta():
	Compute the difference of values with respect to ref_id.

Parameters
----------
values: values to compute from
ref_id: delta values are computed with respect to that reference. It should be valid index or a list of valid
 index from values.
-------
	values: Union[pd.DataFrame, pd.Series]



In [116]:
ast.dump(func.args.args[0].annotation)

"Str(s='Union[pd.DataFrame, pd.Series]')"

In [8]:
import importlib
import inspect

In [19]:
module_name = str(file).replace('\\', '.').replace('.py', '')

In [20]:
module = importlib.import_module(module_name)

In [21]:
signature = inspect.signature(getattr(module, func.name))

In [26]:
for param_name, param_type in signature.parameters.items():
    print(item)

('ast_node', <Parameter "ast_node:Union[_ast.Constant, _ast.Name, _ast.BinOp, _ast.UnaryOp]">)


In [39]:
param_type.annotation

typing.Union[_ast.Constant, _ast.Name, _ast.BinOp, _ast.UnaryOp]

In [45]:
ast.dump(func.args)

"arguments(args=[arg(arg='ast_node', annotation=Subscript(value=Name(id='Union', ctx=Load()), slice=Index(value=Tuple(elts=[Attribute(value=Name(id='ast', ctx=Load()), attr='Constant', ctx=Load()), Attribute(value=Name(id='ast', ctx=Load()), attr='Name', ctx=Load()), Attribute(value=Name(id='ast', ctx=Load()), attr='BinOp', ctx=Load()), Attribute(value=Name(id='ast', ctx=Load()), attr='UnaryOp', ctx=Load())], ctx=Load())), ctx=Load()))], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[])"